# Text Classification:

## Data
<pre>
1. we have total of 20 types of documents(Text files) and total 18828 documents(text files).
2. You can download data from this <a href='https://drive.google.com/open?id=1rxD15nyeIPIAZ-J2VYPrDRZI66-TBWvM'>link</a>, in that you will get documents.rar folder. <br>If you unzip that, you will get total of 18828 documnets. document name is defined as'ClassLabel_DocumentNumberInThatLabel'. 
so from document name, you can extract the label for that document.
4. Now our problem is to classify all the documents into any one of the class.
5. Below we provided count plot of all the labels in our data. 
</pre>

In [1]:
### count plot of all the class labels. 
import os
import seaborn as sns
import tensorflow
import matplotlib.pyplot as plt
import nltk
from nltk.chunk import ne_chunk
from tqdm import tqdm
import codecs
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
%load_ext tensorboard
import tensorflow_addons as tfa


## Assignment:

#### sample document
<pre>
<font color='blue'>
Subject: A word of advice
From: jcopelan@nyx.cs.du.edu (The One and Only)

In article < 65882@mimsy.umd.edu > mangoe@cs.umd.edu (Charley Wingate) writes:
>
>I've said 100 times that there is no "alternative" that should think you
>might have caught on by now.  And there is no "alternative", but the point
>is, "rationality" isn't an alternative either.  The problems of metaphysical
>and religious knowledge are unsolvable-- or I should say, humans cannot
>solve them.

How does that saying go: Those who say it can't be done shouldn't interrupt
those who are doing it.

Jim
--
Have you washed your brain today?
</font>
</pre>

### Preprocessing:
<pre>
useful links: <a href='http://www.pyregex.com/'>http://www.pyregex.com/</a>

<font color='blue'><b>1.</b></font> Find all emails in the document and then get the text after the "@". and then split those texts by '.' 
after that remove the words whose length is less than or equal to 2 and also remove'com' word and then combine those words by space. 
In one doc, if we have 2 or more mails, get all.
<b>Eg:[test@dm1.d.com, test2@dm2.dm3.com]-->[dm1.d.com, dm3.dm4.com]-->[dm1,d,com,dm2,dm3,com]-->[dm1,dm2,dm3]-->"dm1 dm2 dm3" </b> 
append all those into one list/array. ( This will give length of 18828 sentences i.e one list for each of the document). 
Some sample output was shown below. 

> In the above sample document there are emails [jcopelan@nyx.cs.du.edu, 65882@mimsy.umd.edu, mangoe@cs.umd.edu]

preprocessing:
[jcopelan@nyx.cs.du.edu, 65882@mimsy.umd.edu, mangoe@cs.umd.edu] ==> [nyx cs du edu mimsy umd edu cs umd edu] ==> 
[nyx edu mimsy umd edu umd edu]

<font color='blue'><b>2.</b></font> Replace all the emails by space in the original text. 
</pre>

In [0]:
# we have collected all emails and preprocessed them, this is sample output
preprocessed_email

array(['juliet caltech edu',
       'coding bchs edu newsgate sps mot austlcm sps mot austlcm sps mot com  dna bchs edu',
       'batman bmd trw', ..., 'rbdc wsnc org dscomsa desy zeus  desy',
       'rbdc wsnc org morrow stanford edu pangea Stanford EDU',
       'rbdc wsnc org apollo apollo'], dtype=object)

In [0]:
len(preprocessed_email)

18828

<pre>
<font color='blue'><b>3.</b></font> Get subject of the text i.e. get the total lines where "Subject:" occur and remove 
the word which are before the ":" remove the newlines, tabs, punctuations, any special chars.
<b>Eg: if we have sentance like "Subject: Re: Gospel Dating @ \r\r\n" --> You have to get "Gospel Dating"</b> 
Save all this data into another list/array. 

<font color='blue'><b>4.</b></font> After you store it in the list, Replace those sentances in original text by space.

<font color='blue'><b>5.</b></font> Delete all the sentances where sentence starts with <b>"Write to:"</b> or <b>"From:"</b>.
> In the above sample document check the 2nd line, we should remove that

<font color='blue'><b>6.</b></font> Delete all the tags like "< anyword >"
> In the above sample document check the 4nd line, we should remove that "< 65882@mimsy.umd.edu >"


<font color='blue'><b>7.</b></font> Delete all the data which are present in the brackets. 
In many text data, we observed that, they maintained the explanation of sentence 
or translation of sentence to another language in brackets so remove all those.
<b>Eg: "AAIC-The course that gets you HIRED(AAIC - Der Kurs, der Sie anstellt)" --> "AAIC-The course that gets you HIRED"</b>

> In the above sample document check the 4nd line, we should remove that "(Charley Wingate)"


<font color='blue'><b>8.</b></font> Remove all the newlines('\n'), tabs('\t'), "-", "\".

<font color='blue'><b>9.</b></font> Remove all the words which ends with <b>":"</b>.
<b>Eg: "Anyword:"</b>
> In the above sample document check the 4nd line, we should remove that "writes:"


<font color='blue'><b>10.</b></font> Decontractions, replace words like below to full words. 
please check the donors choose preprocessing for this 
<b>Eg: can't -> can not, 's -> is, i've -> i have, i'm -> i am, you're -> you are, i'll --> i will </b>

<b> There is no order to do point 6 to 10. but you have to get final output correctly</b>

<font color='blue'><b>11.</b></font> Do chunking on the text you have after above preprocessing. 
Text chunking, also referred to as shallow parsing, is a task that 
follows Part-Of-Speech Tagging and that adds more structure to the sentence.
So it combines the some phrases, named entities into single word.
So after that combine all those phrases/named entities by separating <b>"_"</b>. 
And remove the phrases/named entities if that is a "Person". 
You can use <b>nltk.ne_chunk</b> to get these. 
Below we have given one example. please go through it. 

useful links: 
<a href='https://www.nltk.org/book/ch07.html'>https://www.nltk.org/book/ch07.html</a>
<a href='https://stackoverflow.com/a/31837224/4084039'>https://stackoverflow.com/a/31837224/4084039</a>
<a href='http://www.nltk.org/howto/tree.html'>http://www.nltk.org/howto/tree.html</a>
<a href='https://stackoverflow.com/a/44294377/4084039'>https://stackoverflow.com/a/44294377/4084039</a>
</pre>

In [0]:
#i am living in the New York
print("i am living in the New York -->", list(chunks))
print(" ")
print("-"*50)
print(" ")
#My name is Srikanth Varma
print("My name is Srikanth Varma -->", list(chunks1))

i am living in the New York --> [('i', 'NN'), ('am', 'VBP'), ('living', 'VBG'), ('in', 'IN'), ('the', 'DT'), Tree('GPE', [('New', 'NNP'), ('York', 'NNP')])]
 
--------------------------------------------------
 
My name is Srikanth Varma --> [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), Tree('PERSON', [('Srikanth', 'NNP'), ('Varma', 'NNP')])]


<pre>We did chunking for above two lines and then We got one list where each word is mapped to a 
POS(parts of speech) and also if you see "New York" and "Srikanth Varma", 
they got combined and represented as a tree and "New York" was referred as "GPE" and "Srikanth Varma" was referred as "PERSON". 
so now you have to Combine the "New York" with <b>"_"</b> i.e "New_York"
and remove the "Srikanth Varma" from the above sentence because it is a person.</pre>

<pre>
<font color='blue'><b>13.</b></font> Replace all the digits with space i.e delete all the digits. 
> In the above sample document, the 6th line have digit 100, so we have to remove that.

<font color='blue'><b>14.</b></font> After doing above points, we observed there might be few word's like
 <b> "_word_" (i.e starting and ending with the _), "_word" (i.e starting with the _),
  "word_" (i.e ending with the _)</b> remove the <b>_</b> from these type of words. 

<font color='blue'><b>15.</b></font>  We also observed some words like <b> "OneLetter_word"- eg: d_berlin, 
"TwoLetters_word" - eg: dr_berlin </b>, in these words we remove the "OneLetter_" (d_berlin ==> berlin) and 
"TwoLetters_" (de_berlin ==> berlin). i.e remove the words 
which are length less than or equal to 2 after spliiting those words by "_". 

<font color='blue'><b>16.</b></font> Convert all the words into lower case and lowe case 
and remove the words which are greater than or equal to 15 or less than or equal to 2.

<font color='blue'><b>17.</b></font> replace all the words except "A-Za-z_" with space. 

<font color='blue'><b>18.</b></font> Now You got Preprocessed Text, email, subject. create a dataframe with those. 
Below are the columns of the df. 
</pre>

In [0]:
data.columns

Index(['text', 'class', 'preprocessed_text', 'preprocessed_subject',
       'preprocessed_emails'],
      dtype='object')


In [0]:
data.iloc[400]

text                    From: arc1@ukc.ac.uk (Tony Curtis)\r\r\r\nSubj...
class                                                         alt.atheism
preprocessed_text       said re is article if followed the quoting rig...
preprocessed_subject                                christian morality is
preprocessed_emails                                   ukc mac macalstr edu
Name: 567, dtype: object


### To get above mentioned data frame --> Try to Write Total Preprocessing steps in One Function Named Preprocess as below. 

In [43]:
def preprocess(Input_Text):
    """Do all the Preprocessing as shown above and
    return a tuple contain preprocess_email,preprocess_subject,preprocess_text for that Text_data"""
    
    pattern=r'([a-zA-Z0-9\-\.]+@[a-zA-z\.]{2,})'
    email_id=re.findall(pattern, Input_Text, flags=re.M|re.I)
    
    email_file=[]
    for ids in email_id:
        e_ids=ids.split('@')[1]
        #print(e_ids)
        domain=e_ids.split('.')
        domain=' '.join(list(d for d in domain if (len(d)>2 and d!='com')))
        email_file.append(domain)
     
    email_file=' '.join(email_file)
    #print(email_file)
    
    #Replacing email with space
    pattern=r'([a-zA-Z0-9\-\.]+@[a-zA-z\.]{2,})'
    Input_Text=re.sub(pattern,' ',Input_Text)
    
    #Input_Text
    
    #Extracting the subject>Removing any word before :>Then remove any character other than a-zA-Z
    pattern_1=r'subject:.+'
    data=re.findall(pattern_1, Input_Text, flags=re.M|re.I)
    pattern_2=r'[a-zA-Z]+:'
    text=re.sub(pattern_2,'',data[0])
    pattern_3=r'[^a-zA-z]'
    text=re.sub(pattern_3,' ',text,flags=re.M|re.I)
    
    subject=text.lower()
    
    #Removing the whole content from the text
    pattern_1=r'(subject:.+)|(From:.+)|(Write to:.+)'
    Input_Text=re.sub(pattern_1,' ',Input_Text)
    
    pattern_3=r'(<.+>)|(\(.+\))|(\(.+\))'
    Input_Text=re.sub(pattern_3,'',Input_Text,flags=re.M|re.I)
    
    pattern_4=r'\b[a-z]+:'
    Input_Text=re.sub(pattern_4,'',Input_Text,flags=re.M|re.I)
    
    pattern_5=r'[\r\n]|[\r\t]'
    Input_Text=re.sub(pattern_5,' ',Input_Text,flags=re.M|re.I)
    
    pattern_6=r'>+|\\+|-+|\\|\"|,|\?'
    Input_Text=re.sub(pattern_6,'',Input_Text,flags=re.M|re.I)
    
    pattern_8=r'\'ve'
    Input_Text=re.sub(pattern_8,' have',Input_Text,flags=re.M|re.I)

    pattern_9=r'can\'t'
    Input_Text=re.sub(pattern_9,'can not',Input_Text,flags=re.M|re.I)

    pattern_10=r'\'s'
    Input_Text=re.sub(pattern_10,'is',Input_Text,flags=re.M|re.I)

    pattern_11=r'i\'ve'
    Input_Text=re.sub(pattern_11,'have',Input_Text,flags=re.M|re.I)

    pattern_11=r'i\'m'
    Input_Text=re.sub(pattern_11,'i am',Input_Text,flags=re.M|re.I)

    pattern_11=r'you\'re'
    Input_Text=re.sub(pattern_11,'you are',Input_Text,flags=re.M|re.I)
    
    pattern_11=r'i\'ll'
    Input_Text=re.sub(pattern_11,'i will',Input_Text,flags=re.M|re.I)
    
    pattern_11=r'\'ll'
    Input_Text=re.sub(pattern_11,' will',Input_Text,flags=re.M|re.I)
   
    pattern_12=r'_[a-zA-Z]+_'
    values=re.findall(pattern_12,Input_Text,flags=re.M|re.I)
    for x in values:
        i=x.split('_')
        i=[x for x in i if len(x)>=2]
        for word in i:
            Input_Text=re.sub(x,word,Input_Text,flags=re.M|re.I)
    
    pattern_12=r'[a-zA-Z]+_ '
    values=re.findall(pattern_12,Input_Text,flags=re.M|re.I)
    for x in values:
        i=x.split('_')
        i=[x for x in i if len(x)>=2]
        for word in i:
            Input_Text=re.sub(x,word+' ',Input_Text,flags=re.M|re.I)
         
    pattern_12=r'[a-zA-Z]{0,2}_[a-zA-Z]+'
    values=re.findall(pattern_12,Input_Text,flags=re.M|re.I)
    for x in values:
        i=x.split('_')
        i=[x for x in i if len(x)>2]
        for word in i:
            if len(i)==0:
                Input_Text=re.sub(x,' ',Input_Text,flags=re.M|re.I)
            else:
                Input_Text=re.sub(x,str(word+' '),Input_Text,flags=re.M|re.I)
    
    tokens=nltk.word_tokenize(Input_Text)
    pos=nltk.pos_tag(tokens)
    chunks=ne_chunk(pos, binary=False)

    named_entities=[]
    person_entities=[]
    for t in chunks.subtrees():
        if t.label() == 'PERSON':
            person_entities.append(list(t))
        
    for entity in person_entities:
        word=[entity[x][0] for x in range(len(entity))]
        pattern=re.escape(str(' '.join(word)))
        #print(pattern)
        Input_Text=re.sub(pattern,'',Input_Text)
    

    for t in chunks.subtrees():
        if t.label() == 'GPE':
            named_entities.append(list(t))
        
    for entity in named_entities:
        word=[entity[x][0] for x in range(len(entity))]
        pattern=re.escape(str(' '.join(word)))
        Input_Text=re.sub(pattern,'_'.join(word),Input_Text)
    
    pattern_12=r'(\b\w{1}\b)|(\w{15,})'
    Input_Text=re.sub(pattern_12,'',Input_Text,flags=re.M|re.I)

    Input_Text=Input_Text.lower()

    pattern_12=r'[^a-zA-Z_]'
    Input_Text=re.sub(pattern_12,' ',Input_Text,flags=re.M|re.I)
    
    pattern_7=r' +'
    Input_Text=re.sub(pattern_7,' ',Input_Text,flags=re.M|re.I)
    
    pattern_11=r' \w{1} '
    Input_Text=re.sub(pattern_11,'',Input_Text,flags=re.M|re.I)
    
    processed_text=Input_Text
    return (email_file,subject,processed_text)

In [44]:
data=open('documents/alt.atheism_49960.txt')
text=data.read()
print(preprocess(text))


('mantis netcom mantis', ' alt atheism  atheist resources', ' alt atheism atheist resources archive atheism resources resources last december atheist resources addresses of atheist organizations usa freedom from religion foundation fish bumper stickers and assorted other atheist paraphernalia are available from the freedom from religion foundation in the us evolution designs evolution designs sell the fish itis fish symbol like the ones stick on their cars but with feet and the word written inside the deluxe mouldedplastic fish is postpaid in the us ca people in the san francisco bay area can get fish from try mailing for net people who go to directly the price is per fish american atheist press aap publish various atheist books critiques of the bible lists of biblical contradictions and so on one such book the bible handbook by and american atheist press pp isbn nd edition bible contradictions absurdities atrocities immoralities contains the bible contradicts itself aap based on the k

In [45]:
data=open('documents/alt.atheism_49960.txt')
text=data.read()
text

'From: mathew <mathew@mantis.co.uk>\nSubject: Alt.Atheism FAQ: Atheist Resources\n\nArchive-name: atheism/resources\nAlt-atheism-archive-name: resources\nLast-modified: 11 December 1992\nVersion: 1.0\n\n                              Atheist Resources\n\n                      Addresses of Atheist Organizations\n\n                                     USA\n\nFREEDOM FROM RELIGION FOUNDATION\n\nDarwin fish bumper stickers and assorted other atheist paraphernalia are\navailable from the Freedom From Religion Foundation in the US.\n\nWrite to:  FFRF, P.O. Box 750, Madison, WI 53701.\nTelephone: (608) 256-8900\n\nEVOLUTION DESIGNS\n\nEvolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones\nChristians stick on their cars, but with feet and the word "Darwin" written\ninside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.\n\nWrite to:  Evolution Designs, 7119 Laurel Canyon #4, North Hollywood,\n           CA 91605.\n\nPeople in the San Francisco Bay area c

In [46]:
folder=os.listdir('documents')

raw_text=[]
email_data=[]
subject_data=[]
processed_text=[]
file_name=[]
class_details=[]

for file in tqdm(folder):
    class_name=file.split('_')[0]
    doc_name=file.split('_')[1]
    
    data=open('documents/'+file)
    input_text=data.read()
    
    email_file,subject,final_text=preprocess(input_text)
    
    raw_text.append(input_text)
    email_data.append(email_file)
    subject_data.append(subject)
    processed_text.append(final_text)
    file_name.append(doc_name)
    class_details.append(class_name)

100%|████████████████████████████████████████████████████████████████████████████| 18828/18828 [38:35<00:00,  8.13it/s]


In [47]:
final_data=[raw_text,
email_data,
subject_data,
processed_text,
file_name,
class_details]

In [48]:
with open('processed_data.pickle','wb') as fe_data_file:
     pickle.dump(final_data, fe_data_file)

In [27]:
with open('processed_data.pickle','rb') as fe_data_file:
    final_data=pickle.load(fe_data_file)

In [29]:
raw_text=final_data[0]
email_data=final_data[1]
subject_data=final_data[2]
processed_text=final_data[3]
file_name=final_data[4]
class_details=final_data[5]

In [32]:
email_data

['mantis co uk netcom com mantis co uk',
 'mantis co uk mantis co uk mantis co uk',
 'dbstu mimsy umd edu cs umd edu',
 'mantis co uk kepler unh edu',
 'Watson Ibm Com harder ccr harder ccr watson ibm com',
 'dbstu batman bmd trw com batman bmd trw com',
 'cco caltech edu jyusenkyou cs jhu edu',
 'dbstu bu edu buphy bu edu',
 'cco caltech edu solntze wpd sgi com',
 'cco caltech edu po CWRU edu',
 'cco caltech edu solntze wpd sgi com',
 'cco caltech edu newton apple com',
 'cco caltech edu jyusenkyou cs jhu edu',
 'cco caltech edu vice ICO TEK COM',
 'cco caltech edu vice ICO TEK COM',
 'cco caltech edu vice ICO TEK COM',
 'vice ICO TEK COM psilink com psilink com vice ICO TEK COM',
 'vice ICO TEK COM monu yoyo cc monash edu au vice ICO TEK COM',
 'vice ICO TEK COM blaze cs jhu edu jyusenkyou cs jhu edu fido asd sgi com solntze wpd sgi com vice ICO TEK COM',
 'pooh bears SUVM SYR EDU SUVM SYR EDU',
 'pooh bears jec MAINE MAINE EDU MAINE MAINE EDU',
 'bmers mantis co uk mantis co uk andr

In [46]:
data=pd.DataFrame(list(zip(raw_text,email_data,subject_data,processed_text,file_name,class_details)),
             columns=['text','email_id','subject','processed_text','file_name','class'])
x=data['email_id']+data['subject']+ data['processed_text']
y=data['class']

In [47]:
total_class=list(data['class'].unique())
class_labels=dict()
for i in range(len(total_class)):
    class_labels[i]=total_class[i]
    y[y==total_class[i]]=i

In [48]:
y = np.asarray(y).astype(np.float32)

### Code checking:

<font color='red' size=4>
After Writing preprocess function. call that functoin with the input text of 'alt.atheism_49960' doc and print the output of the preprocess function
<br>
This will help us to evaluate faster, based on the output we can suggest you if there are any changes.
</font>

### After writing Preprocess function, call the function for each of the document(18828 docs) and then create a dataframe as mentioned above.

### Training The models to Classify: 

<pre>
1. Combine "preprocessed_text", "preprocessed_subject", "preprocessed_emails" into one column. use that column to model. 

2. Now Split the data into Train and test. use 25% for test also do a stratify split. 

3. Analyze your text data and pad the sequnce if required. 
Sequnce length is not restricted, you can use anything of your choice. 
you need to give the reasoning

4. Do Tokenizer i.e convert text into numbers. please be careful while doing it. 
if you are using tf.keras "Tokenizer" API, it removes the <b>"_"</b>, but we need that.

5. code the model's ( Model-1, Model-2 ) as discussed below 
and try to optimize that models.  

6. For every model use predefined Glove vectors. 
<b>Don't train any word vectors while Training the model.</b>

7. Use "categorical_crossentropy" as Loss. 

8. Use <b>Accuracy and Micro Avgeraged F1 score</b> as your as Key metrics to evaluate your model. 

9.  Use Tensorboard to plot the loss and Metrics based on the epoches.

10. Please save your best model weights in to <b>'best_model_L.h5' ( L = 1 or 2 )</b>. 

11. You are free to choose any Activation function, learning rate, optimizer.
But have to use the same architecture which we are giving below.

12. You can add some layer to our architecture but you <b>deletion</b> of layer is not acceptable.

13. Try to use <b>Early Stopping</b> technique or any of the callback techniques that you did in the previous assignments.

14. For Every model save your model to image ( Plot the model) with shapes 
and inlcude those images in the notebook markdown cell, 
upload those imgages to Classroom. You can use "plot_model" 
please refer <a href='https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model'>this</a> if you don't know how to plot the model with shapes. 

</pre>

In [49]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, stratify=y)
x_train.shape

(14121,)

In [65]:
#https://towardsdatascience.com/nlp-preparing-text-for-deep-learning-model-using-tensorflow2-461428138657

In [10]:
import csv
trained_text_embedding=pd.read_csv('glove.6B/glove.6B.100d.txt', sep=' ', header=None,lineterminator='\n',quoting=csv.QUOTE_NONE )

In [37]:
f=[]
for i in trained_text_embedding_chars:
    if type(i)==float:
        f.append(i)
f
# The code had given an error stating the df has an attribute which is float. Now discarding these 3 values.

[nan, nan, nan]

In [39]:
trained_text_embedding=trained_text_embedding.dropna(axis=0)

In [40]:
trained_text_embedding_chars=trained_text_embedding[0]
trained_text_embedding_vector=trained_text_embedding.drop(0,axis=1)

In [41]:
'''Step 1: To create a dictionary with index value: char value
Step 2: Create a matrix where index value has the vector value
Step 3: pre-trained word embeddings matrix'''

text_dict={}
embedding_matrix=np.zeros((len(trained_text_embedding_chars)+1,trained_text_embedding_vector.shape[1]))

for i,text in enumerate(trained_text_embedding_chars.tolist()):
    text_dict[text.lower()]= i
    embedding_matrix[i]=trained_text_embedding_vector.iloc[i]

In [50]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',oov_token='UNK')
tokenizer.fit_on_texts(x_train)

In [51]:
tokenizer.word_index=text_dict
tokenizer.word_index[tokenizer.oov_token] = max(text_dict.values()) + 1

In [52]:
x_train_tokenized=tokenizer.texts_to_sequences(x_train)
x_test_tokenized=tokenizer.texts_to_sequences(x_test)

In [53]:
max_values_in_list=[]
for j in x_train_tokenized:
    value=max(j)
    max_values_in_list.append(value)
    
input_dim=max(max_values_in_list)
input_dim

399997

In [54]:
padded_x_train=tf.keras.preprocessing.sequence.pad_sequences(x_train_tokenized,maxlen=(len(tokenizer.word_index) + 1), padding='pre')
padded_x_test=tf.keras.preprocessing.sequence.pad_sequences(x_test_tokenized,maxlen=(len(tokenizer.word_index) + 1), padding='pre')

MemoryError: Unable to allocate 21.0 GiB for an array with shape (14121, 399999) and data type int32

In [36]:
print(padded_x_train.shape)
print(padded_x_test.shape)

(14121, 64337)
(4707, 64337)


### Model-1: Using 1D convolutions with word embeddings

<pre>
<b>Encoding of the Text </b> --> For a given text data create a Matrix with Embedding layer as shown Below. 
In the example we have considered d = 5, but in this assignment we will get d = dimension of Word vectors we are using.
 i.e if we have maximum of 350 words in a sentence and embedding of 300 dim word vector, 
 we result in 350*300 dimensional matrix for each sentance as output after embedding layer
<img src='https://i.imgur.com/kiVQuk1.png'>
Ref: https://i.imgur.com/kiVQuk1.png

<b>Reference:</b>
<a href='https://stackoverflow.com/a/43399308/4084039'>https://stackoverflow.com/a/43399308/4084039</a>
<a href='https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/'>https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/</a>

<b><a href='https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work'>How EMBEDDING LAYER WORKS </a></b>

</pre>

### Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [37]:
#Clearing Logs before running the Model
import shutil
shutil.rmtree('logs')

In [4]:
#https://keras.io/examples/nlp/pretrained_word_embeddings/
shutil.

In [38]:
#Building the model
input_shape=(padded_x_train.shape[1])#input is just of this dimension
inputs = tf.keras.Input(input_shape)

conv_m=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,input_shape))(inputs) #Input shape=(Batch_size, Size of input)
conv_m=tf.keras.layers.Conv1D(10,5,strides=2,activation='relu',input_shape=(1,100), kernel_initializer=tf.keras.initializers.HeUniform(10))(conv_m)

conv_n=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,input_shape))(inputs) #Input shape=(Batch_size, Size of input)
conv_n=tf.keras.layers.Conv1D(20,5,strides=2,activation='relu',input_shape=(1,100), kernel_initializer=tf.keras.initializers.HeUniform(20))(conv_n)

conv_o=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,input_shape))(inputs) #Input shape=(Batch_size, Size of input)
conv_o=tf.keras.layers.Conv1D(40,5,strides=2,activation='relu',input_shape=(1,100), kernel_initializer=tf.keras.initializers.HeUniform(30))(conv_o)

output_1= tf.keras.layers.concatenate([conv_m,conv_n,conv_o])

output_1=tf.keras.layers.MaxPool1D()(output_1)

conv_i=tf.keras.layers.Conv1D(10,7,strides=2,activation='relu',input_shape=output_1.shape, kernel_initializer=tf.keras.initializers.HeUniform(40))(output_1)

conv_j=tf.keras.layers.Conv1D(20,7,strides=2,activation='relu',input_shape=output_1.shape, kernel_initializer=tf.keras.initializers.HeUniform(50))(output_1)

conv_k=tf.keras.layers.Conv1D(40,7,strides=2,activation='relu',input_shape=output_1.shape, kernel_initializer=tf.keras.initializers.HeUniform(60))(output_1)

output_2= tf.keras.layers.concatenate([conv_i,conv_j,conv_k])

output_2=tf.keras.layers.MaxPool1D()(output_2)
output_2=tf.keras.layers.Conv1D(35,7,strides=2,activation='relu', kernel_initializer=tf.keras.initializers.HeUniform(150))(output_2)
output_2=tf.keras.layers.Flatten()(output_2)
output_2=tf.keras.layers.Dropout(0.2)(output_2)
output_2=tf.keras.layers.Dense(20, activation = 'softmax')(output_2)

model=tf.keras.Model(inputs=inputs,outputs=output_2)

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64337)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64337, 100)   6433700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 64337, 100)   6433700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 64337, 100)   6433700     input_1[0][0]                    
______________________________________________________________________________________________

In [39]:
early_stopping=tf.keras.callbacks.EarlyStopping(patience=3,monitor='accuracy',min_delta=0.001)
tensorboard=tf.keras.callbacks.TensorBoard(log_dir='logs/model_1')

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy',tfa.metrics.F1Score(num_classes=20, 
                      average='micro',
                      threshold=0.5)], )
model.fit(padded_x_train,y_train,epochs=50,callbacks=[early_stopping,tensorboard],validation_split=0.2)


Epoch 1/50
 14/353 [>.............................] - ETA: 1:05:37 - loss: 3.0077 - accuracy: 0.0670

KeyboardInterrupt: 

In [15]:
model.save('models/model_1')

INFO:tensorflow:Assets written to: models/model_1\assets


In [6]:
#%tensorboard --logdir {'./logs'}  --host localhost

In [17]:
model.predict(padded_x_test)

ValueError: in user code:

    C:\Anaconda3\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Anaconda3\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Anaconda3\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
    C:\Anaconda3\lib\site-packages\keras\engine\training.py:1537 predict_step
 

<img src='https://i.imgur.com/fv1GvFJ.png'>
ref: 'https://i.imgur.com/fv1GvFJ.png'

<pre>
1. all are Conv1D layers with any number of filter and filter sizes, there is no restriction on this.

2. use concatenate layer is to concatenate all the filters/channels. 

3. You can use any pool size and stride for maxpooling layer.

4. Don't use more than 16 filters in one Conv layer becuase it will increase the no of params. 
( Only recommendation if you have less computing power )

5. You can use any number of layers after the Flatten Layer.
</pre>

### Model-2 : Using 1D convolutions with character embedding

<pre>
<pre><img src="https://i.ytimg.com/vi/CNY8VjJt-iQ/maxresdefault.jpg" width="70%">
Here are the some papers based on Char-CNN
 1. Xiang Zhang, Junbo Zhao, Yann LeCun. <a href="http://arxiv.org/abs/1509.01626">Character-level Convolutional Networks for Text Classification</a>.NIPS 2015
 2. Yoon Kim, Yacine Jernite, David Sontag, Alexander M. Rush. <a href="https://arxiv.org/abs/1508.06615">Character-Aware Neural Language Models</a>. AAAI 2016
 3. Shaojie Bai, J. Zico Kolter, Vladlen Koltun. <a href="https://arxiv.org/pdf/1803.01271.pdf">An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling</a>
 4. Use the pratrained char embeddings <a href='https://github.com/minimaxir/char-embeddings/blob/master/glove.840B.300d-char.txt'>https://github.com/minimaxir/char-embeddings/blob/master/glove.840B.300d-char.txt</a>
</pre>

<img src='https://i.imgur.com/EuuoJtr.png'>

In [ ]:
pretrained_char_embed=pd.read_csv('Character_Embedding.txt', delimiter=' ', columns=None)


In [ ]:
#Building the model
input_shape=(7544)#input is just of this dimension
inputs = tf.keras.Input(input_shape)

conv_m=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,7544))(inputs) #Input shape=(Batch_size, Size of input)
conv_m=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=(1,100))(conv_m)

conv_n=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,7544))(inputs) #Input shape=(Batch_size, Size of input)
conv_n=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=(1,100))(conv_n)

conv_o=tf.keras.layers.Embedding(input_dim=(input_dim+1),output_dim=100,input_shape=(1,7544))(inputs) #Input shape=(Batch_size, Size of input)
conv_o=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=(1,100))(conv_o)

output_1= tf.keras.layers.concatenate([conv_m,conv_n,conv_o])

output_1=tf.keras.layers.MaxPool1D()(output_1)

conv_i=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=output_1.shape)(output_1)

conv_j=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=output_1.shape)(output_1)

conv_k=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu',input_shape=output_1.shape)(output_1)

output_2= tf.keras.layers.concatenate([conv_i,conv_j,conv_k])

output_2=tf.keras.layers.MaxPool1D()(output_2)
output_2=tf.keras.layers.Conv1D(1,8,strides=2,activation='relu')(output_2)
output_2=tf.keras.layers.Flatten()(output_2)
output_2=tf.keras.layers.Dropout(0.2)(output_2)
output_2=tf.keras.layers.Dense(20, activation = 'softmax')(output_2)

model=tf.keras.Model(inputs=inputs,outputs=output_2)

model.summary()

